# Firefighters

## Load the data

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('../data/Firefighters.csv')  # load data set
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 5 columns):
Race        118 non-null object
Position    118 non-null object
Oral        118 non-null float64
Written     118 non-null int64
Combine     118 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.7+ KB


## Test passers, top scorers

Originally there were 8 Lieutenant and 7 Captain positions and within two years total of 16 Lieutenant and 8 Captain positions.

In [2]:
def is_top_scoring(x,thresh_ltn,thresh_cpt):
    return 1 if ((x[0]=='Captain' and x[1] >= thresh_cpt) or (x[0]=='Lieutenant' and x[1] >= thresh_ltn)) else 0

def get_top_scoring(n_ltn, n_cpt):
    thresh_ltn = min(data.groupby('Position')['Combine'].nlargest(n_ltn+2)['Captain'])
    thresh_cpt = min(data.groupby('Position')['Combine'].nlargest(n_cpt+2)['Lieutenant'])
    return data[['Position','Combine']].apply(is_top_scoring, args=(thresh_ltn,thresh_cpt), axis=1)

passed = data['Combine'].apply(lambda x: int(x >= 70)).rename('Passed')
top_scoring = get_top_scoring(8, 7).rename('Top_k')
top_scoring2 = get_top_scoring(16, 8).rename('Top_k_2y')
data = pd.concat([top_scoring, top_scoring2, passed, data], axis=1)

data.head(3)

,Top_k,Top_k_2y,Passed,Race,Position,Oral,Written,Combine
0,1,1,1,W,Captain,89.52,95,92.808
1,1,1,1,W,Captain,80.00,95,89.000
2,1,1,1,W,Captain,82.38,87,85.152


## Logistic regression


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

features = data.columns.drop(['Race', 'Position', 'Top_k', 'Top_k_2y', 'Passed']).values
targets = ['Top_k', 'Top_k_2y', 'Passed']

# One-Hot encoding for categorical data
for key in ['Position', 'Race']:
    dummies = pd.get_dummies(data[key]).rename(columns=lambda x: key+'_' + str(x))
    data = pd.concat([data, dummies], axis=1)

# Training loop
predicted_dict = {}
for pos in ['Lieutenant','Captain']:
    data2 = data[data['Position'] == pos]
    X, X_test, y, y_test = train_test_split(data2.iloc[:,1:], data2[targets], test_size=0.4, random_state=12345)
    predicted = pd.concat([X_test['Race'], X_test['Position']], axis=1)
    for key in targets:
        # Train
        lr = LogisticRegression(solver='lbfgs', max_iter=500) 
        lr.fit(X[features], y[key])
        y_pred = lr.predict(X_test[features])
        # Results
        predicted[key] = y_pred
    predicted_dict[pos] = predicted.copy()

In [4]:
predicted_dict['Lieutenant'].head(3)

,Race,Position,Top_k,Top_k_2y,Passed
45,W,Lieutenant,1,1,1
101,W,Lieutenant,0,0,0
61,W,Lieutenant,0,0,1


In [5]:
predicted_dict['Captain'].head(3)

,Race,Position,Top_k,Top_k_2y,Passed
32,H,Captain,0,0,0
19,W,Captain,0,0,1
2,W,Captain,1,1,1


## Adverse impact ratio for predicted data


In [6]:
def get_summary(df):
    passed = df.groupby(['Race','Passed']).size().unstack().fillna(0)
    top_k = df.groupby(['Race','Top_k']).size().unstack().fillna(0)
    top_k_2y = df.groupby(['Race','Top_k_2y']).size().unstack().fillna(0)
    summary =  pd.DataFrame({
        'Race': ['B','H','W'], 
        'Pass': [passed[1]['B'],passed[1]['H'],passed[1]['W']],
        'Pass_pr': [0,0,0],
        'Fail': [passed[0]['B'],passed[0]['H'],passed[0]['W']],
        'Total': [0,0,0],
        'Top_k': [top_k[1]['B'],top_k[1]['H'],top_k[1]['W']],
        'Top_k_2y': [top_k_2y[1]['B'],top_k_2y[1]['H'],top_k_2y[1]['W']],
    })
    summary['Total'] = summary['Pass'] + summary['Fail']
    summary['Pass_pr'] = 100*summary['Pass']/summary['Total']
    summary['Adverse_impact'] = (100*summary['Pass_pr']/max(summary['Pass_pr']))
    summary['Pass_pr'] = summary['Pass_pr']
    return summary

### Lieutenant

#### Original data

In [7]:
pd.options.display.float_format = '{:,.2f}'.format
get_summary(data[data['Position'] == 'Lieutenant'])

,Race,Pass,Pass_pr,Fail,Total,Top_k,Top_k_2y,Adverse_impact
0,B,6,31.58,13,19,0.00,4.00,54.32
1,H,3,20.00,12,15,0.00,0.00,34.40
2,W,25,58.14,18,43,12.00,16.00,100.00


#### Predicted data

In [8]:
get_summary(predicted_dict['Lieutenant'])

,Race,Pass,Pass_pr,Fail,Total,Top_k,Top_k_2y,Adverse_impact
0,B,2,40.00,3,5,0.00,1.00,76.00
1,H,2,28.57,5,7,0.00,0.00,54.29
2,W,10,52.63,9,19,4.00,4.00,100.00


### Captain

#### Original data

In [9]:
get_summary(data[data['Position'] == 'Captain'])

,Race,Pass,Pass_pr,Fail,Total,Top_k,Top_k_2y,Adverse_impact
0,B,3,37.50,5,8,0.00,0.00,58.59
1,H,3,37.50,5,8,0.00,0.00,58.59
2,W,16,64.00,9,25,3.00,3.00,100.00


#### Predicted data

In [10]:
get_summary(predicted_dict['Captain'])

,Race,Pass,Pass_pr,Fail,Total,Top_k,Top_k_2y,Adverse_impact
0,B,1,50.00,1,2,0.00,0.00,71.43
1,H,2,40.00,3,5,0.00,0.00,57.14
2,W,7,70.00,3,10,2.00,2.00,100.00
